In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import warnings

warnings.filterwarnings("ignore")

In [2]:
# read csv
df = pd.read_csv("historical_transaction.csv")

df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,-0.247824,6.425853,-6.485658,-8.241462,NaN,0.03%,Tuesday,-9.808905,4.753455,2.537819,...,0.592543,14.967527,6.691044,-0.275572,2.518542,4.637431,-14.296510,1.898115,-1.225228,0
1,2.623274,-2.808263,5.321746,-2.437653,0.614041,0.02%,Wednesday,10.710053,1.608882,-2.027541,...,-1.712788,-3.638637,-2.256492,2.703101,1.760729,2.660614,1.182922,-3.203078,0.711018,0
2,-5.460551,5.997701,-4.614194,-6.035128,-1.986438,-0.0%,Tuesday,-3.364649,-2.493473,1.429913,...,2.206956,7.292558,10.095556,1.601155,1.710645,-12.519434,-5.261707,2.032666,-6.445528,0
3,1.566307,-0.080517,0.064754,6.063193,-2.253075,0.0%,Thurday,-6.111982,-1.796377,-3.169510,...,-3.841669,5.189840,10.291069,-3.642396,0.674300,-1.104317,-3.922342,-0.613947,7.275179,0
4,0.750715,15.656095,-6.778600,11.251037,-0.308200,0.0%,Wednesday,2.498892,-0.323229,-2.624680,...,8.933781,-7.295794,-11.246059,-4.876594,3.395415,-2.646844,-5.273267,-5.259536,-3.240523,0


In [3]:
X = df[df.columns[:-1]]
y = df['y']

In [4]:
# check data types
list(X.dtypes)

[dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dty

In [5]:
# inspect features with object data type
obj_columns = X.select_dtypes(include='object').columns
X[obj_columns]

# x5:  some kind of percentage, should be converted to float?
# x6:  day of the week (categorical), should be label encoded and then one-hot encoded.
# x20: months of the year (categorical), should be label encoded and then one-hot encoded.
# x49: true/false, should be converted to 0s and 1s.
# x57: some kind of amount, should be converted to float?

,x5,x6,x20,x27,x49,x57
0,0.03%,Tuesday,Aug,Afternoon,False,$1578.34
1,0.02%,Wednesday,Jul,Afternoon,False,$767.76
2,-0.0%,Tuesday,Aug,Evening,False,$931.06
3,0.0%,Thurday,Aug,Afternoon,True,$2124.52
4,0.0%,Wednesday,Jul,Evening,False,$-3055.67
...,...,...,...,...,...,...
7995,0.01%,Tuesday,Jun,Afternoon,False,$-1664.04
7996,0.0%,Tuesday,May,Afternoon,False,$-42.81
7997,-0.01%,Wednesday,Aug,Afternoon,True,$220.59
7998,-0.0%,Tuesday,Aug,Evening,True,$467.47


In [6]:
# look at categorical values
print(pd.unique(X['x6']))
print(pd.unique(X['x20']))
print(pd.unique(X['x27']))
print(pd.unique(X['x49']))

['Tuesday' 'Wednesday' 'Thurday' nan 'Monday' 'Friday']
['Aug' 'Jul' 'Jun' 'May' 'Apr' 'Sept' 'Oct' nan 'Mar' 'Feb' 'Nov' 'Dec'
 'Jan']
['Afternoon' 'Evening' 'Night' 'Morning' nan]
[False True nan]


In [7]:
# convert x5 to float
X['x5'] = X['x5'].str.replace('%', '').astype(float)
X['x5']

0       0.03
1       0.02
2      -0.00
3       0.00
4       0.00
        ... 
7995    0.01
7996    0.00
7997   -0.01
7998   -0.00
7999   -0.02
Name: x5, Length: 8000, dtype: float64

In [8]:
# convert x57 to numeric
X['x57'] = X['x57'].str.replace('$', '').astype(float)
X['x57']

0       1578.34
1        767.76
2        931.06
3       2124.52
4      -3055.67
         ...   
7995   -1664.04
7996     -42.81
7997     220.59
7998     467.47
7999    -598.35
Name: x57, Length: 8000, dtype: float64

In [9]:
# fetch features with object data type
obj_columns = X.select_dtypes(include='object').columns
X[obj_columns]

,x6,x20,x27,x49
0,Tuesday,Aug,Afternoon,False
1,Wednesday,Jul,Afternoon,False
2,Tuesday,Aug,Evening,False
3,Thurday,Aug,Afternoon,True
4,Wednesday,Jul,Evening,False
...,...,...,...,...
7995,Tuesday,Jun,Afternoon,False
7996,Tuesday,May,Afternoon,False
7997,Wednesday,Aug,Afternoon,True
7998,Tuesday,Aug,Evening,True


In [10]:
# look at null values in categorical features
X[obj_columns].isnull().sum()

x6      91
x20    100
x27     80
x49     92
dtype: int64

In [11]:
# inspect value counts in categorical features
for column in obj_columns:
    print(column)
    print(X[column].value_counts())
    print()
    
# due to clear dominent values, we can use mode to fill null values
# other methods can be used to deal with x20 since the mode is not very dominent in comparison to other features

x6
Wednesday    4135
Tuesday      2347
Thurday      1179
Monday        199
Friday         49
Name: x6, dtype: int64

x20
Jun     1912
Jul     1823
May     1289
Aug     1229
Apr      635
Sept     545
Mar      210
Oct      165
Feb       47
Nov       26
Dec       10
Jan        9
Name: x20, dtype: int64

x27
Evening      3895
Afternoon    3553
Night         272
Morning       200
Name: x27, dtype: int64

x49
False    4330
True     3578
Name: x49, dtype: int64



In [12]:
# fill null values with the mode in categorical features
for column in obj_columns:
    X[column].fillna(X[column].mode()[0], inplace=True)

X[obj_columns]

,x6,x20,x27,x49
0,Tuesday,Aug,Afternoon,False
1,Wednesday,Jul,Afternoon,False
2,Tuesday,Aug,Evening,False
3,Thurday,Aug,Afternoon,True
4,Wednesday,Jul,Evening,False
...,...,...,...,...
7995,Tuesday,Jun,Afternoon,False
7996,Tuesday,May,Afternoon,False
7997,Wednesday,Aug,Afternoon,True
7998,Tuesday,Aug,Evening,True


In [13]:
# convert x49 to binary
X['x49'] = X['x49'].astype(bool).astype(int)
X['x49']

0       0
1       0
2       0
3       1
4       0
       ..
7995    0
7996    0
7997    1
7998    1
7999    0
Name: x49, Length: 8000, dtype: int32

In [14]:
# fetch features with object data type
obj_columns = X.select_dtypes(include='object').columns

In [15]:
# one-hot encode all categorical features
X = pd.get_dummies(data=X, columns=obj_columns)

In [16]:
# confirm that all datatypes are numeric now
dtypes = np.unique(X.dtypes)
dtypes

array([dtype('uint8'), dtype('int32'), dtype('float64')], dtype=object)

In [17]:
# see null values for different datatypes
# we should only see null values in float64 type features since we have already dealt with null values in all other features
for dtype in dtypes:
    print(dtype)
    print(X[X.select_dtypes(include=dtype).columns].isnull().sum())
    print()

uint8
x6_Friday        0
x6_Monday        0
x6_Thurday       0
x6_Tuesday       0
x6_Wednesday     0
x20_Apr          0
x20_Aug          0
x20_Dec          0
x20_Feb          0
x20_Jan          0
x20_Jul          0
x20_Jun          0
x20_Mar          0
x20_May          0
x20_Nov          0
x20_Oct          0
x20_Sept         0
x27_Afternoon    0
x27_Evening      0
x27_Morning      0
x27_Night        0
dtype: int64

int32
x49    0
dtype: int64

float64
x0     84
x1     76
x2     74
x3     66
x4     81
       ..
x95    70
x96    87
x97    86
x98    77
x99    77
Length: 96, dtype: int64



In [18]:
# fill null values with mean in all features
# this will not at all change non-float features since they have already been dealt with and have no null values
X.fillna(X.mean(), inplace=True)
# confirm all null values have been dealt with
X.isnull().sum().max()

0

In [19]:
# normalize all features using min-max scaling
# note that binary columns stay as binary after normalization
scaler = MinMaxScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_normalized

,x0,x1,x2,x3,x4,x5,x7,x8,x9,x10,...,x20_Jun,x20_Mar,x20_May,x20_Nov,x20_Oct,x20_Sept,x27_Afternoon,x27_Evening,x27_Morning,x27_Night
0,0.518612,0.486436,0.384060,0.379905,0.447052,0.875,0.363094,0.826459,0.583404,0.518486,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.601313,0.394552,0.595169,0.475059,0.500422,0.750,0.731973,0.647202,0.447905,0.572824,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.368461,0.482175,0.417521,0.416078,0.276050,0.500,0.478945,0.413346,0.550522,0.404265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.570867,0.421694,0.501177,0.614431,0.253045,0.500,0.429555,0.453084,0.414012,0.479772,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.547374,0.578281,0.378823,0.699486,0.420850,0.500,0.584357,0.537061,0.430183,0.818428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.448987,0.388385,0.408050,0.519767,0.171111,0.625,0.299322,0.569450,0.433170,0.521980,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7996,0.588380,0.373617,0.414128,0.152990,0.447052,0.500,0.534641,0.456124,0.682308,0.539315,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7997,0.590477,0.388884,0.471986,0.392960,0.389005,0.375,0.658408,0.687927,0.618631,0.587235,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7998,0.485355,0.222314,0.566264,0.705778,0.506176,0.500,0.462054,0.456851,0.128519,0.489422,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [20]:
# inspect if y values are skewed
y.value_counts()

# there is a huge imbalance in the dataset as almost 95% of the dataset has y value of 0.

# techniques we can use to overcome this imbalance:

# partial resampling: i will use this technique to avoid overfitting of data. I will aim for a 75-25 split.

# oversampling for class 1: will probably result in overfitting due to incredibly high 95-5 imbalance:
        
# undersampling for class 0: might lead to loss of important information:
    
# combination of oversampling and undersampling: i will use this technique:
#     This will avoid overfitting and loss of information to some extent, and should help in a better training dataset for out models
#     I will be first apply random undersampling to bring down imbalance from 95-5 to 90-10
#     Next, I will use ADASYN for oversampling to bring down imbalance from 90-10 to 80-20 

0    7561
1     439
Name: y, dtype: int64

In [21]:
# perform random undersampling to target 10-90 ratio
undersample_ratio = 10/90

random_undersample = RandomUnderSampler(sampling_strategy=undersample_ratio, random_state=0)
X_normalized_undersampled, y_undersampled = random_undersample.fit_resample(X_normalized, y)
y_undersampled.value_counts()

0    3951
1     439
Name: y, dtype: int64

In [22]:
# perform ADASYN oversampling to target 20-80 ratio
oversample_ratio = 25/75

adasyn_oversample = ADASYN(sampling_strategy=oversample_ratio, random_state=0)
X_normalized_undersampled_oversampled, y_undersampled_oversampled = adasyn_oversample.fit_resample(X_normalized_undersampled, y_undersampled)
y_undersampled_oversampled.value_counts()

0    3951
1    1253
Name: y, dtype: int64

In [23]:
# splitting data into train and test datasets
# due to 80-20 imbalancing, random splitting will not be optimal for a good model.
# i will use stratified sampling to ensure that the proportions of both classes are preserved

X_train, X_test, y_train, y_test = train_test_split(X_normalized_undersampled_oversampled, y_undersampled_oversampled, stratify=y_undersampled_oversampled, test_size=0.2)
print(y_train.value_counts())
print(y_test.value_counts())

0    3161
1    1002
Name: y, dtype: int64
0    790
1    251
Name: y, dtype: int64


In [24]:
# insepct current transactions
current = pd.read_csv('current_transaction.csv')
current

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99
0,-1.514284,-12.485595,10.593498,-5.002425,-0.419907,0.01%,Wednesday,-1.089788,0.196689,2.139182,...,4.0,7.152219,9.629630,2.544992,-4.702698,-0.387781,-8.510972,-1.320433,3.479030,-11.769354
1,-0.988268,29.655347,3.838360,3.868849,-1.294265,-0.0%,Wednesday,-3.944761,-1.565021,8.216759,...,3.0,-9.673933,-5.787857,-17.676507,-0.318778,-2.078193,3.499461,5.576143,-2.698396,-9.410448
2,3.317484,-4.822399,9.678680,17.815120,0.754625,-0.0%,Wednesday,-0.148101,-1.645103,2.135212,...,3.0,-10.733762,-0.305876,-6.785229,0.220154,2.496943,-3.883966,6.226439,2.454146,-1.224857
3,4.863474,-0.006963,5.931868,-4.529761,-2.261027,-0.0%,Wednesday,-17.570001,2.168551,3.732319,...,3.0,8.062077,3.949675,0.494529,-5.162874,1.159392,-2.661473,-2.462430,1.572517,-8.811929
4,-2.870043,-9.920092,6.925923,3.860836,0.252619,-0.01%,Wednesday,0.857456,-2.186101,-0.502004,...,4.0,-0.210101,-9.162798,4.953034,2.455752,-0.617239,7.565136,6.612028,-3.757768,1.030277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.261316,-2.081898,6.033927,3.689495,NaN,0.01%,Wednesday,9.140968,-3.040429,1.099079,...,3.0,3.710152,7.631708,-16.155056,0.053846,-1.802197,10.356600,1.877766,-4.785522,8.387005
1996,-4.546886,8.391715,2.808590,17.929822,-2.314800,0.0%,Wednesday,-4.851376,0.247683,3.277210,...,3.0,-1.013693,-6.021149,-1.043471,-6.921365,0.564669,-1.237992,9.876203,1.778705,0.778693
1997,2.539207,15.453591,-0.120234,5.700297,-0.990693,-0.02%,Tuesday,-1.430567,1.010819,4.438095,...,3.0,11.165736,-11.940162,2.416564,0.753879,2.334901,4.826179,15.464218,1.700007,-1.163875
1998,-0.303316,10.489227,10.601150,-3.361656,-2.060652,-0.01%,Thurday,14.110021,0.510631,-1.371320,...,3.0,-3.052423,14.436015,-6.665361,-0.914924,0.253017,-9.592686,4.215946,9.952626,11.139543


In [26]:
# inspect null values
current.isnull().sum()

x0     15
x1     27
x2     22
x3     24
x4     19
       ..
x95    27
x96    20
x97    20
x98    22
x99    14
Length: 100, dtype: int64

In [27]:
# inspect non-numeric features
current.select_dtypes(include='object')

,x5,x6,x20,x27,x49,x57
0,0.01%,Wednesday,Jul,Afternoon,True,$103.77
1,-0.0%,Wednesday,Jun,Evening,False,$-1284.96
2,-0.0%,Wednesday,May,Morning,False,$2908.7
3,-0.0%,Wednesday,Jul,Afternoon,False,$838.78
4,-0.01%,Wednesday,Mar,Afternoon,True,$-792.86
...,...,...,...,...,...,...
1995,0.01%,Wednesday,Aug,Evening,False,$1524.46
1996,0.0%,Wednesday,May,Afternoon,NaN,$1091.15
1997,-0.02%,Tuesday,Apr,Afternoon,False,$-277.0
1998,-0.01%,Thurday,Jul,Evening,False,$-2274.46
